# Scrapping de Tripadvisor

In [1]:
from bs4 import BeautifulSoup
import urllib.request, re, pandas
import csv
from random import randint
from time import sleep
from time import time
import numpy as np
import math

Voici le résultat attendu et les différentes données à récupérer pour chaque restaurant:

In [2]:
df_total = pandas.DataFrame(columns = ["Resto","Adresse","Code Postal","Ville",'Note',"Date du commentaire",'Commentaire'])
df_total

,Resto,Adresse,Code Postal,Ville,Note,Date du commentaire,Commentaire


On récupère la liste des restaurants.

In [6]:
df_total_page = pandas.read_csv("liens restaurants - tripadvisor.csv", sep=";")
df_total_page = df_total_page.drop('Unnamed: 0', 1)
print (df_total_page.shape)
df_total_page.head()

(13359, 3)


,nom,avis,url
0,Epicure,1,https://www.tripadvisor.fr/Restaurant_Review-g...
1,Pur' - Jean-François Rouquette,1,https://www.tripadvisor.fr/Restaurant_Review-g...
2,ALKARAM,1,https://www.tripadvisor.fr/Restaurant_Review-g...
3,Fromagerie Danard,1,https://www.tripadvisor.fr/Restaurant_Review-g...
4,Il Etait Un Square,1,https://www.tripadvisor.fr/Restaurant_Review-g...


## Fonctions

On commence par récupérer les liens de chaque restaurant et les informations sommaires des pages qui les listent.

In [104]:
# on va sur la page principale
main_url = "https://www.tripadvisor.fr/Restaurants-g187147-Paris_Ile_de_France.html#MAINWRAP"
main_furl = urllib.request.urlopen(main_url)
main_soup = BeautifulSoup(main_furl, 'html.parser')

In [8]:
# on récupère le nombre de pages de restaurants
def get_nb_pages(soup):
    x = soup.find("span", class_="pageNum current")
    if x == None:
        return 1
    for i in x.next_siblings:
        if "'STANDARD_PAGINATION', 'last'" in str(i):
            nb_pages = int(i["data-page-number"])
    return nb_pages

In [9]:
# on récupère le nombre de restaurants
def get_nb_resto(soup):
    x = main_soup.find("div", class_="popIndex popIndexDefault")
    x = str(x)
    a = x.index("sur") + 4
    b = x.index("Restaurants") - 1
    x = x[a:b]
    x = int(x.replace("\xa0", ""))
    return x

In [10]:
# on récupère le lien pour toutes les pages de restaurants
def get_url_pages_resto(nb_pages):
    liste_page_resto = ["https://www.tripadvisor.fr/Restaurants-g187147-Paris_Ile_de_France.html#MAINWRAP"]
    for n in range (1, nb_pages):
        index_page = 30 * n
        url_page = "https://www.tripadvisor.fr/RestaurantSearch-g187147-oa" + str(index_page) + "-Paris_Ile_de_France.html#EATERY_LIST_CONTENTS"
        liste_page_resto.append(url_page)
    return liste_page_resto

In [11]:
# on récupère le nom, la présence de commentaire et le lien de chaque restaurant de la page
def scrap_main(soup):
    x = soup.find_all("h3", class_="title")
    nom = []
    avis = []
    url = []
    for i in x:
        # pour chaque resto on récupère son nom
        name = i.next_element.next_element.next_element.strip()
        nom.append(name)
        # s'il a des commentaires ou non
        com = str(i.next_sibling.next_sibling.next_sibling.next_sibling)
        if "Soyez le premier à donner votre avis sur ce restaurant" in com:
            avis.append(0)
        else:
            avis.append(1)
        # et son lien html
        lien = "https://www.tripadvisor.fr" + str(i.next_element.next_element["href"])
        url.append(lien)
    nom = pandas.Series(nom)
    avis = pandas.Series(avis)
    url = pandas.Series(url)
    df = pandas.concat( (nom, avis, url), axis=1)
    df.columns = ["nom", "avis", "url"]
    return df

On récupère les différentes informations relatives au restaurant sur la page du restaurant.

In [42]:
# on récupère la note du restaurant
def get_note(soup):
    x = soup.find("img", property="ratingValue")
    note = float(x["content"])
    return note

In [107]:
# on récupère le nombre de commentaires du restaurant
def get_nb_com(soup):
    x = soup.find("a", property="reviewCount")
    if x == None:
        z = soup.find("span", class_="tabs_pers_counts")
        nb_com = int(z.next_element.replace("(", "").replace(")", ""))
    else:
        nb_com = int(x["content"])
    return nb_com

In [14]:
# on récupère le lien de chaque page de commentaires du restaurant
def get_url_pages_com(url, nb_pages, soup):
    liste_page_com = [url]
    if nb_pages != 1:
        x = soup.find("a", class_="pageNum taLnk")
        lien = "https://www.tripadvisor.fr" + x["href"]
        l = lien.split("-or10-")
        for n in range(1, nb_pages):
            index_page = 10 * n
            index_page = "-or" + str(index_page) + "-"
            url_page = l[0] + index_page + l[1]
            liste_page_com.append(url_page)
    return liste_page_com

In [83]:
# on récupère l'adresse du restaurant
def get_adresse(soup):
    x = soup.find("span", property="streetAddress")
    if x.next_element.string is None:
        z = soup.find("span", class_="extended-address")
        if z == None:
            adresse = None
        else:
            adresse = z.next_element
    else:
        adresse = x.next_element
    x = soup.find("span", property="postalCode")
    code_postal = x.next_element
    x = soup.find("span", property="addressLocality")
    ville = x.next_element
    return adresse, code_postal, ville

On définit un timer pour rendre notre procédure moins brutale.

In [16]:
def r_sleep():
    ''' generates a random sleep between 1.000 and 2.000 seconds '''
    length = float(randint(1000, 2000)) / 1000
    sleep(length)

On convertit nos dates.

In [17]:
def conversion_date(date):
    l = date.split(" ")
    mois = {"janvier":"01", "février":"02", "mars":"03", "avril":"04", "mai":"05", "juin":"06", "juillet":"07", "août":"08", "septembre":"09", "octobre":"10", "novembre":"11", "décembre":"12"}
    if len(l[0]) == 1:
        l[0] = "0" + l[0]
    res = "/".join([l[0], mois[l[1]], l[2]])
    return res

In [18]:
def comparaison_date(date, reference):
    """date = 'dd/mm/aaaa' et reference = aaaammdd"""
    l = date.split("/")
    res = l[2] + l[1] + l[0]
    res = int(res)
    if res > reference:
        return True
    else:
        return False

In [19]:
x = "15 avril 2015"
x = conversion_date(x)
comparaison_date(x, 20160101)

False

On teste si une valeur est manquante ou pas.

In [20]:
def test_nan(x):
    return x != x

On récupère les informations de base relatives à tous les restaurants (nom, url)

In [21]:
def get_df_liens_restaurants(): 
    # on va sur la page principale
    main_url = "https://www.tripadvisor.fr/Restaurants-g187147-Paris_Ile_de_France.html#MAINWRAP"
    main_furl = urllib.request.urlopen(main_url)
    main_soup = BeautifulSoup(main_furl, 'html.parser')

    # on récupère quelques informations de base
    nb_resto = get_nb_resto(main_soup)
    nb_pages = get_nb_pages(main_soup)
    liste_page_resto = get_url_pages_resto(nb_pages)

    # on créé un dataframe contenant les informations présentes sur les pages principales
    df_total_page = pandas.DataFrame(columns=["nom", "avis", "url"])

    # on fait une boucle sur toutes les pages
    s = 1
    for url_page in liste_page_resto:
        # on temporise chaque requête
        r_sleep()
        furl_page = urllib.request.urlopen(url_page)
        soup_page = BeautifulSoup(furl_page, 'html.parser')
        # on récupère les informations présentent sur la page
        df_page = scrap_main(soup_page)
        df_total_page = pandas.concat( (df_total_page, df_page) )
        print ("Page ", s, "/", nb_pages, " terminée")
        s = s + 1

    # on met en forme notre dataframe avant de l'enregistrer
    df_total_page.reset_index(drop=True, inplace=True)
    df_total_page.to_csv("liens restaurants - tripadvisor.csv", sep=";")
    return df_total_page

On récupère toutes les informations qui nous intéresse sur le restaurant.

In [105]:
url_mort = ["https://www.tripadvisor.fr/Restaurant_Review-g187147-d10253116-Reviews-Apsaras-Paris_Ile_de_France.html", "https://www.tripadvisor.fr/Restaurant_Review-g187147-d10253386-Reviews-Herve_at_Galerie_Lafayette-Paris_Ile_de_France.html"]

def get_all(nb_premier_resto=1, nb_dernier_resto=100, df_total_page=df_total_page):
    
    debut = time()

    # on initialise notre base de données
    df_total = pandas.DataFrame(columns = ["Resto","Adresse","Code Postal","Ville",'Note',"Date du commentaire",'Commentaire'])

    # on récupère les liens des restaurants qui nous intéressent
    df_restreint = df_total_page[df_total_page.avis==1]
    liste_url_restaurant = df_restreint["url"]
    liste_nom_restaurant = df_restreint["nom"]
    
    # on fait une boucle sur tous les restaurants
    nb_resto = nb_dernier_resto - nb_premier_resto + 1, 
    n = nb_premier_resto
    for i in range(nb_premier_resto-1, nb_dernier_resto):
        url_resto = liste_url_restaurant[i]
        nom_resto = liste_nom_restaurant[i]
        
        # on temporise notre processus
        #r_sleep()
        
        print ("###################")
        print ("Restaurant", n, "/", nb_dernier_resto, ":")
        print (nom_resto)
        print (url_resto)
        
        # on ne prend pas en compte certains liens morts
        if url_resto not in url_mort:
            furl_resto = urllib.request.urlopen(url_resto)
            soup_resto = BeautifulSoup(furl_resto, 'html.parser')

            # on vérifie si le restaurant est encore ouvert
            if "FERMÉ" not in soup_resto.find("h1", id="HEADING").next_element.next_element.next_element.strip():

                # on récupère des informations de base sur la page du restaurant
                #note = get_note(soup_resto)
                nb_com = get_nb_com(soup_resto)
                nb_pages = get_nb_pages(soup_resto)
                url_pages_com = get_url_pages_com(url_resto, nb_pages, soup_resto)
                adresse, code_postal, ville = get_adresse(soup_resto)

                # on créé un dataframe relatif au restaurant
                df_resto = pandas.DataFrame(index=range(0, get_nb_com(soup_resto)), columns=["Resto","Adresse","Code Postal","Ville",'Note',"Date du commentaire",'Commentaire'])
                df_resto["Resto"] = nom_resto
                adresse, code_postal, ville = get_adresse(soup_resto)
                df_resto["Adresse"] = adresse
                df_resto["Code Postal"] = code_postal
                df_resto["Ville"] = ville

                # on fait une boucle sur toutes les pages de commentaires du restaurant
                p = 0
                point_break = 1000000000
                for url_page in url_pages_com:

                    # on temporise notre processus
                    #r_sleep()
                    furl_page = urllib.request.urlopen(url_page)
                    soup_page = BeautifulSoup(furl_page, 'html.parser')

                    # on récupère les dates de publication des commentaires de la page
                    s = p
                    x = soup_page.find_all("span", class_="ratingDate")
                    for i in x:
                        if (s < p + 10) and (s < point_break):
                            date = i.next_element.strip()
                            if "Avis publié:" in date:
                                date = i["title"]
                            else:
                                date = date[14:]
                            date = conversion_date(date)
                            if comparaison_date(date, 20160101):
                                df_resto.at[s, "Date du commentaire"] = date
                                s = s + 1
                            else:
                                point_break = s
                                print ("-------- Scraping commentaire stoppé: commentaire antérieur au 01/01/2016")
                                break

                    # on récupère les notes des commentaires de la page
                    s = p
                    x = soup_page.find_all("span", class_="rate sprite-rating_s rating_s")
                    for i in x:
                        if (s < p + 10) and (s < point_break):
                            df_resto.at[s, "Note"] = int(i.next_element.next_element["alt"][0])
                            s = s + 1

                    # on récupère les commentaires de la page
                    s = p
                    x = soup_page.find_all("div", class_="entry")
                    for i in x:
                        if (s < p + 10) and (s < point_break):
                            df_resto.at[s, "Commentaire"] = i.next_element.next_element.next_element
                            s = s + 1
                    print ("----- Page commentaire ", int((p/10)+1), "/", nb_pages, " terminée")
                    p = p + 10
                    if point_break != 1000000000:
                        break

                # on rajoute les informations du restaurant (sans les nan) à notre base de données
                df_resto = df_resto[df_resto.Commentaire == df_resto.Commentaire]
                df_total = pandas.concat( (df_total, df_resto) ) 
                print ("Restaurant ", n, "/", nb_dernier_resto, " terminée")
                n = n + 1

            else:
                print ("***** Scraping stoppé: restaurant fermé ! *****")
        
        else:
            print ("***** Scraping stoppé: lien url mort ! *****")
        
    # on enregistre notre résultat
    path = "Data/" + str(nb_premier_resto) + "_to_" + str(nb_dernier_resto) + " - tripadvisor.csv"
    df_total.to_csv(path, sep=";")
    fin = time()
    duree = fin - debut
    print ("########################################")
    print ("Le processus a mis ", duree, " secondes")
    print ("\n", "\n")
    
    return df_total

## Algorithme

On récupère les liens des restaurants dans un dataframe.

In [20]:
df_total_page = get_df_liens_restaurants()

Page  1 / 446  terminée
Page  2 / 446  terminée
Page  3 / 446  terminée
Page  4 / 446  terminée
Page  5 / 446  terminée
Page  6 / 446  terminée
Page  7 / 446  terminée
Page  8 / 446  terminée
Page  9 / 446  terminée
Page  10 / 446  terminée
Page  11 / 446  terminée
Page  12 / 446  terminée
Page  13 / 446  terminée
Page  14 / 446  terminée
Page  15 / 446  terminée
Page  16 / 446  terminée
Page  17 / 446  terminée
Page  18 / 446  terminée
Page  19 / 446  terminée
Page  20 / 446  terminée
Page  21 / 446  terminée
Page  22 / 446  terminée
Page  23 / 446  terminée
Page  24 / 446  terminée
Page  25 / 446  terminée
Page  26 / 446  terminée
Page  27 / 446  terminée
Page  28 / 446  terminée
Page  29 / 446  terminée
Page  30 / 446  terminée
Page  31 / 446  terminée
Page  32 / 446  terminée
Page  33 / 446  terminée
Page  34 / 446  terminée
Page  35 / 446  terminée
Page  36 / 446  terminée
Page  37 / 446  terminée
Page  38 / 446  terminée
Page  39 / 446  terminée
Page  40 / 446  terminée
Page  41 

In [24]:
df_total_page.head()

,nom,avis,url
0,Epicure,1,https://www.tripadvisor.fr/Restaurant_Review-g...
1,Pur' - Jean-François Rouquette,1,https://www.tripadvisor.fr/Restaurant_Review-g...
2,ALKARAM,1,https://www.tripadvisor.fr/Restaurant_Review-g...
3,Fromagerie Danard,1,https://www.tripadvisor.fr/Restaurant_Review-g...
4,Il Etait Un Square,1,https://www.tripadvisor.fr/Restaurant_Review-g...


In [25]:
df_total_page.shape

(13359, 3)

Parmi ces restaurants, seuls ceux avec au minimum un commentaire nous intéressent.

In [27]:
df_total_page[df_total_page.avis == 1].shape[0]

12454

On récupère les informations présentes sur la page de chaque restaurant ayant reçu des avis.

In [89]:
for i in range(9401, 12400, 100):
    i_fin = i + 99
    df_total = get_all(i, i_fin, df_total_page)

###################
Restaurant 9401 / 9500 :
Taglia Osteria
https://www.tripadvisor.fr/Restaurant_Review-g187147-d8857220-Reviews-Taglia_Osteria-Paris_Ile_de_France.html
----- Page commentaire  1 / 1  terminée
Restaurant  9401 / 9500  terminée
###################
Restaurant 9402 / 9500 :
Columbus Cafe
https://www.tripadvisor.fr/Restaurant_Review-g187147-d1035646-Reviews-Columbus_Cafe-Paris_Ile_de_France.html
-------- Scraping commentaire stoppé: commentaire antérieur au 01/01/2016
----- Page commentaire  1 / 2  terminée
Restaurant  9402 / 9500  terminée
###################
Restaurant 9403 / 9500 :
La Commedia
https://www.tripadvisor.fr/Restaurant_Review-g187147-d5706630-Reviews-La_Commedia-Paris_Ile_de_France.html
-------- Scraping commentaire stoppé: commentaire antérieur au 01/01/2016
----- Page commentaire  1 / 1  terminée
Restaurant  9403 / 9500  terminée
###################
Restaurant 9404 / 9500 :
Hokkaido
https://www.tripadvisor.fr/Restaurant_Review-g187147-d2336784-Reviews-Hokk

On récupère les derniers restaurants.

In [109]:
df_total = get_all(12401, 12454)

###################
Restaurant 12401 / 12454 :
La Paillote d'Or
https://www.tripadvisor.fr/Restaurant_Review-g187147-d7695605-Reviews-La_Paillote_d_Or-Paris_Ile_de_France.html
----- Page commentaire  1 / 1  terminée
Restaurant  12401 / 12454  terminée
###################
Restaurant 12402 / 12454 :
Speed Rabbit Pizza
https://www.tripadvisor.fr/Restaurant_Review-g187147-d7695265-Reviews-Speed_Rabbit_Pizza-Paris_Ile_de_France.html
-------- Scraping commentaire stoppé: commentaire antérieur au 01/01/2016
----- Page commentaire  1 / 1  terminée
Restaurant  12402 / 12454  terminée
###################
Restaurant 12403 / 12454 :
La Nouvelle Couronne d'Orient
https://www.tripadvisor.fr/Restaurant_Review-g187147-d6665770-Reviews-La_Nouvelle_Couronne_d_Orient-Paris_Ile_de_France.html
-------- Scraping commentaire stoppé: commentaire antérieur au 01/01/2016
----- Page commentaire  1 / 1  terminée
Restaurant  12403 / 12454  terminée
###################
Restaurant 12404 / 12454 :
cafe frances
https: